In [1]:
import pandas as pd

# Load the dataset
file_path = "../datasets/combined.parquet"
data = pd.read_parquet(file_path)

# Display the first few rows of the dataset for a quick overview
data.head()

,sentence,is_sarcastic
0,thirtysomething scientists unveil doomsday clo...,1.0
1,dem rep. totally nails why congress is falling...,0.0
2,eat your veggies: 9 deliciously different recipes,0.0
3,inclement weather prevents liar from getting t...,1.0
4,mother comes pretty close to using word 'strea...,1.0


In [2]:
# Checking for any null values in the dataset
null_check = data.isnull().sum()

# Checking the distribution of the 'is_sarcastic' column
label_distribution = data["is_sarcastic"].value_counts(normalize=True)

null_check, label_distribution

(sentence        0
 is_sarcastic    0
 dtype: int64,
 is_sarcastic
 0.0    0.521391
 1.0    0.478609
 Name: proportion, dtype: float64)

In [3]:
import re
from sklearn.model_selection import train_test_split

# Data cleaning: removing special characters and escape sequences from the sentences
data["sentence"] = data["sentence"].apply(lambda x: re.sub(r"[\n\r\t]+", " ", x))

# Splitting the dataset into training, validation, and testing sets
train_data, test_data = train_test_split(data, test_size=0.3, random_state=42)
val_data, test_data = train_test_split(test_data, test_size=0.5, random_state=42)

# Showing the size of each split
train_size, val_size, test_size = len(train_data), len(val_data), len(test_data)
train_size, val_size, test_size

(28322, 6069, 6070)

In [4]:
from transformers import BertTokenizer
from torch.utils.data import Dataset, DataLoader
import torch

# Initialize the BERT tokenizer
tokenizer = BertTokenizer.from_pretrained("bert-base-uncased")


class SarcasticSentencesDataset(Dataset):
    """
    A custom PyTorch Dataset for the sarcastic sentences dataset.
    """

    def __init__(self, sentences, labels, tokenizer, max_len):
        self.sentences = sentences
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.sentences)

    def __getitem__(self, item):
        sentence = str(self.sentences[item])
        label = self.labels[item]

        # Encoding the sentences using the tokenizer
        encoding = self.tokenizer.encode_plus(
            sentence,
            add_special_tokens=True,  # Add '[CLS]' and '[SEP]'
            max_length=self.max_len,
            return_token_type_ids=False,
            padding="max_length",
            return_attention_mask=True,
            return_tensors="pt",  # Return PyTorch tensors
            truncation=True,
        )

        return {
            "sentence": sentence,
            "input_ids": encoding["input_ids"].flatten(),
            "attention_mask": encoding["attention_mask"].flatten(),
            "labels": torch.tensor(label, dtype=torch.long),
        }


# Constants
MAX_LEN = 128  # Maximum length of the tokens list
BATCH_SIZE = 16

# Creating instances of the SarcasticSentencesDataset
train_dataset = SarcasticSentencesDataset(
    train_data["sentence"].to_numpy(),
    train_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

val_dataset = SarcasticSentencesDataset(
    val_data["sentence"].to_numpy(),
    val_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

test_dataset = SarcasticSentencesDataset(
    test_data["sentence"].to_numpy(),
    test_data["is_sarcastic"].to_numpy(),
    tokenizer,
    MAX_LEN,
)

# Creating the DataLoaders for training, validation, and testing
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

# Checking the first batch from the train_loader
next(iter(train_loader))

/Users/remikalbe/.pyenv/versions/3.10.4/envs/iit_dl_project/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'sentence': ["you will never love anything as much as dc's panda loves snow",
  "the currys' nursery for their second baby is freaking adorable",
  '10 ways to deal with a difficult coworker',
  'increasingly anxious man worried order confirmation email never going to come',
  'learning resilience from a master',
  "dunno... feind seems to be a german surname. perhaps a feindslayer is someone who goes around killing people named feind? i'm happy being viewed as a fiend, however. not so easily slayed! ;)",
  'Yea, i got one about the UK. Let me find it, hold on.',
  'man turns vegetarian for 36 hours',
  "this is the fall checklist your home's been waiting for",
  'How about, "I think it\'s a good idea"?As Jito said, maybe YOU can\'t, but I guess that\'s your problem.',
  'study links meat, sugar consumption to early death among those who choose to be happy in life',
  "Among other things, generally adaptions to a colder environment.   As for your theory as a whole, doesn't the whole 1

In [5]:
import torch
from torch.utils.data import DataLoader
from transformers import (
    BertForSequenceClassification,
    AdamW,
    get_linear_schedule_with_warmup,
)
from torch.nn import CrossEntropyLoss
from tqdm import tqdm  # for displaying progress
import numpy as np

In [6]:
device = torch.device(
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

In [7]:
from typing import Dict
import numpy as np
import torch
from torch.utils.data import DataLoader
from torch.nn import Module, CrossEntropyLoss
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from tqdm import tqdm


def train_epoch(
    model: Module,
    data_loader: DataLoader,
    optimizer: Optimizer,
    device: torch.device,
    scheduler: _LRScheduler,
    loss_fn: CrossEntropyLoss,
    n_examples: int,
) -> Dict[str, float]:
    model.train()
    losses = []
    correct_predictions = 0

    # Initialize counters
    tp_sarcasm = 0
    tn_non_sarcasm = 0
    fp_sarcasm = 0
    fn_sarcasm = 0

    for batch in tqdm(data_loader, total=len(data_loader)):
        input_ids = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels = batch["labels"].to(device)

        optimizer.zero_grad()

        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits

        loss = loss_fn(logits, labels)
        losses.append(loss.item())
        loss.backward()
        optimizer.step()
        scheduler.step()

        _, preds = torch.max(logits, dim=1)
        correct_predictions += torch.sum(preds == labels)

        # Update TP, TN, FP, FN counters
        tp_sarcasm += torch.sum((preds == 1) & (labels == 1)).item()
        tn_non_sarcasm += torch.sum((preds == 0) & (labels == 0)).item()
        fp_sarcasm += torch.sum((preds == 1) & (labels == 0)).item()
        fn_sarcasm += torch.sum((preds == 0) & (labels == 1)).item()

    # Compute precision and recall for sarcasm
    precision_sarcasm = tp_sarcasm / (tp_sarcasm + fp_sarcasm + 1e-10)
    recall_sarcasm = tp_sarcasm / (tp_sarcasm + fn_sarcasm + 1e-10)

    # Compute precision and recall for non-sarcasm
    precision_non_sarcasm = tn_non_sarcasm / (tn_non_sarcasm + fn_sarcasm + 1e-10)
    recall_non_sarcasm = tn_non_sarcasm / (tn_non_sarcasm + fp_sarcasm + 1e-10)

    return {
        "accuracy": correct_predictions.float() / n_examples,
        "precision_sarcasm": precision_sarcasm,
        "recall_sarcasm": recall_sarcasm,
        "precision_non_sarcasm": precision_non_sarcasm,
        "recall_non_sarcasm": recall_non_sarcasm,
        "loss": np.mean(losses),
    }

In [8]:
def eval_model(
    model: Module,
    data_loader: DataLoader,
    device: torch.device,
    loss_fn: CrossEntropyLoss,
    n_examples: int,
) -> Dict[str, float]:
    model.eval()
    losses = []
    correct_predictions = 0

    # Initialize counters
    tp_sarcasm = 0
    tn_non_sarcasm = 0
    fp_sarcasm = 0
    fn_sarcasm = 0

    with torch.no_grad():
        for batch in tqdm(data_loader, total=len(data_loader)):
            input_ids = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels = batch["labels"].to(device)

            outputs = model(input_ids=input_ids, attention_mask=attention_mask)
            logits = outputs.logits

            loss = loss_fn(logits, labels)
            losses.append(loss.item())

            _, preds = torch.max(logits, dim=1)
            correct_predictions += torch.sum(preds == labels)

            # Update TP, TN, FP, FN counters
            tp_sarcasm += torch.sum((preds == 1) & (labels == 1)).item()
            tn_non_sarcasm += torch.sum((preds == 0) & (labels == 0)).item()
            fp_sarcasm += torch.sum((preds == 1) & (labels == 0)).item()
            fn_sarcasm += torch.sum((preds == 0) & (labels == 1)).item()

    # Compute precision and recall for sarcasm
    precision_sarcasm = tp_sarcasm / (tp_sarcasm + fp_sarcasm + 1e-10)
    recall_sarcasm = tp_sarcasm / (tp_sarcasm + fn_sarcasm + 1e-10)

    # Compute precision and recall for non-sarcasm
    precision_non_sarcasm = tn_non_sarcasm / (tn_non_sarcasm + fn_sarcasm + 1e-10)
    recall_non_sarcasm = tn_non_sarcasm / (tn_non_sarcasm + fp_sarcasm + 1e-10)

    return {
        "accuracy": correct_predictions.float() / n_examples,
        "precision_sarcasm": precision_sarcasm,
        "recall_sarcasm": recall_sarcasm,
        "precision_non_sarcasm": precision_non_sarcasm,
        "recall_non_sarcasm": recall_non_sarcasm,
        "loss": np.mean(losses),
    }

In [9]:
# Load pre-trained model
model = BertForSequenceClassification.from_pretrained(
    "bert-base-uncased",
    num_labels=2,
    hidden_dropout_prob=0.3,  # dropout rate,
    attention_probs_dropout_prob=0.3,  # dropout rate in attention heads
)
model.to(device)  # Send the model to GPU if available

# Optimizer
optimizer = AdamW(
    model.parameters(), lr=2e-5, weight_decay=0.01  # weight decay for regularization
)

num_epochs = 3

# Total number of training steps
total_steps = len(train_loader) * num_epochs

# Scheduler for learning rate
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=0, num_training_steps=total_steps
)

# Loss function
loss_fn = CrossEntropyLoss()

# Training and Validation
for epoch in range(num_epochs):
    print(f"Epoch {epoch + 1}/{num_epochs}")
    print("-" * 10)

    # Training phase
    train_output = train_epoch(
        model, train_loader, optimizer, device, scheduler, loss_fn, len(train_dataset)
    )

    print(f"Training Metrics:")
    train_metrics = [
        f"Loss: {train_output['loss']:.4f}",
        f"Accuracy: {train_output['accuracy']:.4f}",
        f"Sarcasm Precision: {train_output['precision_sarcasm']:.4f}",
        f"Sarcasm Recall: {train_output['recall_sarcasm']:.4f}",
        f"Non-Sarcasm Precision: {train_output['precision_non_sarcasm']:.4f}",
        f"Non-Sarcasm Recall: {train_output['recall_non_sarcasm']:.4f}",
    ]
    print(" | ".join(train_metrics))

    # Validation phase
    val_output = eval_model(model, val_loader, device, loss_fn, len(val_dataset))

    print(f"Validation Metrics:")
    val_metrics = [
        f"Loss: {val_output['loss']:.4f}",
        f"Accuracy: {val_output['accuracy']:.4f}",
        f"Sarcasm Precision: {val_output['precision_sarcasm']:.4f}",
        f"Sarcasm Recall: {val_output['recall_sarcasm']:.4f}",
        f"Non-Sarcasm Precision: {val_output['precision_non_sarcasm']:.4f}",
        f"Non-Sarcasm Recall: {val_output['recall_non_sarcasm']:.4f}",
    ]
    print(" | ".join(val_metrics))

    print()

# Save the model
torch.save(model.state_dict(), "sarcastic_model.pth")

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
/Users/remikalbe/.pyenv/versions/3.10.4/envs/iit_dl_project/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch 1/3
----------


100%|██████████| 1771/1771 [08:50<00:00,  3.34it/s]


Training Metrics:
Loss: 0.4276 | Accuracy: 0.7969 | Sarcasm Precision: 0.7885 | Sarcasm Recall: 0.7866 | Non-Sarcasm Precision: 0.8046 | Non-Sarcasm Recall: 0.8064


100%|██████████| 380/380 [00:30<00:00, 12.36it/s]


...

Validation Metrics:
Loss: 0.3703 | Accuracy: 0.8448 | Sarcasm Precision: 0.9021 | Sarcasm Recall: 0.7611 | Non-Sarcasm Precision: 0.8054 | Non-Sarcasm Recall: 0.9229

Epoch 3/3
----------


100%|██████████| 1771/1771 [09:03<00:00,  3.26it/s]


Training Metrics:
Loss: 0.2534 | Accuracy: 0.8900 | Sarcasm Precision: 0.8833 | Sarcasm Recall: 0.8873 | Non-Sarcasm Precision: 0.8962 | Non-Sarcasm Recall: 0.8924


100%|██████████| 380/380 [00:32<00:00, 11.59it/s]


Validation Metrics:
Loss: 0.3695 | Accuracy: 0.8479 | Sarcasm Precision: 0.9131 | Sarcasm Recall: 0.7570 | Non-Sarcasm Precision: 0.8044 | Non-Sarcasm Recall: 0.9328

